<a href="https://colab.research.google.com/github/BBlopusy/machinelearningstudies/blob/main/knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KNN

In [63]:
import pandas as pd
import numpy as np
from scipy.spatial import KDTree #ogarnie nam algorytm KNN
from sklearn.metrics import mean_absolute_error #średni błąd bezwzgledny, policzymy dzięki
#temu o ile w naszym przypadku w dolarach są błędne prognozy 
import random



## Pobranie danych i ich przygotowanie
**!wget** to linuksowe polecenie ściągające plik z sieci

In [64]:
!wget https://github.com/betacord/ML/raw/master/lab1/data/data.csv

--2023-04-05 19:56:09--  https://github.com/betacord/ML/raw/master/lab1/data/data.csv
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/betacord/ML/master/lab1/data/data.csv [following]
--2023-04-05 19:56:09--  https://raw.githubusercontent.com/betacord/ML/master/lab1/data/data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7381541 (7.0M) [text/plain]
Saving to: ‘data.csv.4’

data.csv.4          100%[===================>]   7.04M  --.-KB/s    in 0.04s   

2023-04-05 19:56:09 (201 MB/s) - ‘data.csv.4’ saved [7381541/7381541]



### Wczytanie danych 

In [65]:
houses = pd.read_csv('/content/data.csv')

In [66]:
houses

,AppraisedValue,SqFtLot,WaterSystem,SewerSystem,Access,Topography,StreetSurface,RestrictiveSzShape,InadequateParking,PcntUnusable,...,NativeGrowthProtEsmt,Easements,OtherDesignation,DeedRestrictions,DevelopmentRightsPurch,WaterProblems,TranspConcurrency,OtherProblems,lat,long
0,619000.0,6542712,1,1,4,0,1,0,0,0,...,0,0,0,0,1,0,0,0,47.354019,-122.225166
1,708000.0,105850,2,1,3,0,2,0,0,0,...,0,0,0,0,0,0,0,0,47.611248,-122.053154
2,297000.0,1667,2,2,4,0,1,0,2,0,...,0,0,0,0,0,0,0,0,47.701248,-122.365784
3,523000.0,1872,2,2,4,0,1,0,2,0,...,0,0,0,0,0,0,0,0,47.537918,-122.392845
4,288000.0,3480,2,2,4,0,1,0,0,0,...,0,0,0,0,0,0,0,0,47.510227,-122.182167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52415,523000.0,6982,2,2,4,0,1,0,0,0,...,0,0,0,0,0,0,0,0,47.769810,-122.367447
52416,425000.0,5481,2,2,4,0,1,0,2,0,...,0,0,0,0,0,0,0,0,47.510895,-122.366936
52417,377000.0,6793,2,2,3,0,1,0,0,0,...,0,0,0,0,0,0,0,0,47.766289,-122.358192
52418,581000.0,7200,2,2,4,0,1,0,0,0,...,0,0,0,0,0,0,0,0,47.769989,-122.364822


Wyświetlenie kolumn i wczytanie ich w czystej postaci tablicy numphy:

In [67]:
houses.columns.values

array(['AppraisedValue', 'SqFtLot', 'WaterSystem', 'SewerSystem',
       'Access', 'Topography', 'StreetSurface', 'RestrictiveSzShape',
       'InadequateParking', 'PcntUnusable', 'MtRainier', 'Olympics',
       'Cascades', 'Territorial', 'SeattleSkyline', 'PugetSound',
       'LakeWashington', 'LakeSammamish', 'SmallLakeRiverCreek',
       'OtherView', 'WfntLocation', 'WfntFootage', 'WfntBank',
       'WfntPoorQuality', 'WfntRestrictedAccess', 'WfntAccessRights',
       'WfntProximityInfluence', 'TidelandShoreland', 'LotDepthFactor',
       'TrafficNoise', 'PowerLines', 'OtherNuisances', 'NbrBldgSites',
       'Contamination', 'AdjacentGolfFairway', 'AdjacentGreenbelt',
       'HistoricSite', 'CurrentUseDesignation', 'NativeGrowthProtEsmt',
       'Easements', 'OtherDesignation', 'DeedRestrictions',
       'DevelopmentRightsPurch', 'WaterProblems', 'TranspConcurrency',
       'OtherProblems', 'lat', 'long'], dtype=object)

### Selekcja atrybutów

Selekcja atrybutów, wybieramy te które nas interesują ⛱ 
Czyli wartość, wielkość działki, system nawadniający itd.
My wybierzemy dokładnie za radą ekspertów: wielkość działki i współrzędne geograficzne. Nie wpływają w sposób liniowy. Będziemy próbować przewidzieć wartość czyli AppraisedValue na podstawie SqFtlot i współrzednych geograficznych czyli lat i long. 


In [68]:
# wyodrębnienie atrybutu decyzyjnego
values = houses['AppraisedValue']
# usuwamy go z systemu decyjnego, tworząc w ten sposób system informacyjny
houses.drop('AppraisedValue',1, inplace=True) 
#wskazujemy czy chcemy usunąć po osi wierszy czy po osi kolumn, my wskazujemy że po osi kolumn
#dlatego dajemy w parametrze axis wartość: 1
#nie chcemy zwróconej kopi tej ramki tylko żeby została zastąpiona oryginalna parametrem inplance =True


<ipython-input-68-e2790199a246>:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  houses.drop('AppraisedValue',1, inplace=True)


In [69]:
houses

,SqFtLot,WaterSystem,SewerSystem,Access,Topography,StreetSurface,RestrictiveSzShape,InadequateParking,PcntUnusable,MtRainier,...,NativeGrowthProtEsmt,Easements,OtherDesignation,DeedRestrictions,DevelopmentRightsPurch,WaterProblems,TranspConcurrency,OtherProblems,lat,long
0,6542712,1,1,4,0,1,0,0,0,0,...,0,0,0,0,1,0,0,0,47.354019,-122.225166
1,105850,2,1,3,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,47.611248,-122.053154
2,1667,2,2,4,0,1,0,2,0,0,...,0,0,0,0,0,0,0,0,47.701248,-122.365784
3,1872,2,2,4,0,1,0,2,0,0,...,0,0,0,0,0,0,0,0,47.537918,-122.392845
4,3480,2,2,4,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,47.510227,-122.182167
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52415,6982,2,2,4,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,47.769810,-122.367447
52416,5481,2,2,4,0,1,0,2,0,0,...,0,0,0,0,0,0,0,0,47.510895,-122.366936
52417,6793,2,2,3,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,47.766289,-122.358192
52418,7200,2,2,4,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,47.769989,-122.364822


### **Normalizacja danych**
Mamy różne atrybuty i różne wartości np. w SqFtLot mamy duże i małe wartości, rozsrtrzelone mocno. Musimy przetransformować dane, pozmieniać wartości żeby różnice były mniejsze, żeby rozkłady prawdopodobieństwa do siebie się upodobniły ale jednocześnie, żeby zależności miedzy nimi dalej były te same.

In [70]:
houses = (houses - houses.mean()) / (houses.max() - houses.min()) 
#pomnijszamy przez wartość średnią i dzielimy przez różnicę wartości maksymalną i minimalną w całej ramce danych
houses = houses[['lat', 'long', 'SqFtLot']] #zostawiamy sobie te trzy atrybuty które nas interesują

In [71]:
houses

,lat,long,SqFtLot
0,-0.002260,-0.002427,0.926695
1,0.004289,-0.001374,0.012250
2,0.006580,-0.003288,-0.002550
3,0.002422,-0.003454,-0.002521
4,0.001717,-0.002164,-0.002293
...,...,...,...
52415,0.008325,-0.003298,-0.001795
52416,0.001734,-0.003295,-0.002008
52417,0.008236,-0.003241,-0.001822
52418,0.008330,-0.003282,-0.001764


### Podział na trenig i test
Trening 80% test 20% 


In [72]:
test_rows = random.sample(houses.index.tolist(), int(round(len(houses)*.2)))
train_rows = set(range(len(houses))) - set(test_rows) 

In [73]:
df_test = houses.loc[test_rows]
df_train = houses.drop(test_rows)

In [74]:
test_values = values.loc[test_rows] #indekser loc jak wyżej
train_values = values.loc[train_rows]

<ipython-input-74-5fb2b1cbe0e7>:2: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  train_values = values.loc[train_rows]


liczebność danych treningowych:

In [75]:
len(train_values)

41936

liczebność danych testowych:

In [76]:
len(test_values)

10484

## KNN implementujemy algorytm
Jak wejdziemy w źródło tego algorytmu klasy KDTree to jest tam odległość Minkowskiego, matematyka nauczyciela Einsteina 🦾

In [77]:
kdtree = KDTree(df_train)

In [78]:
def predict(query_point, k):#query_point czyli dla konkretnego wiersza z naszej bazy będziemy przydzielać predykcję
  _, idx = kdtree.query(query_point, k) #ta metoda zwraca nam dwie wartości 'd' i 'i'
  # gdzie d to tablica odległości do najbliższych obiektów, natomiast 'i' jest indeksami najbliższych sąsiadów
  # w tym przypadku nie korzystamy z pierwszej wartości, dajemy podkreślnik
  return np.mean(train_values.iloc[idx])

In [79]:
train_predicted_values = [] #pusta lista, miejsce na wartości które będą przewidywane
train_actual_values = []

#wstawimy na sztywno 5 najbliższych sąsiadów, później będziemy optymalizować
for _id, row in df_train.iterrows():
  prediction = predict(row, 5) 
  actual_value = train_values[_id]

  train_predicted_values.append(prediction)
  train_actual_values.append(actual_value)

właściwy wynik:

In [80]:
mae = mean_absolute_error(train_values, train_predicted_values)
f'Średni błąd bezwględny: {mae} $ '

'Średni błąd bezwględny: 60055.38398988936 $ '

😾48 minut 
